In [56]:
import glob
import torch
import json
from PIL import Image
import random
import pandas as pd

In [57]:
data_dir = "../data/rockfish_full/"
model_name = "x152" # from predict._MODELS
device = torch.device("cpu")

crops_dir = "./x152_crops/"

In [58]:
pths = glob.glob(data_dir + "**/*.pth", recursive=True)
pths = [p for p in list(pths) if model_name in p]
len(pths)

62

In [59]:
table = [] # filepath

num_images = 0
num_boxes = 0
num_valid = 0
num_invalid = 0
empty_images = 0

for p in pths:
    loaded = torch.load(p, map_location=device)
    for im in loaded:
        im = random.choice(loaded)
        img = Image.open(im['file_name'])
        filename = im['file_name'].replace("../data/", "")
        boxes = im['instances'].pred_boxes
        confs = im['instances'].scores.tolist()
        num_boxes += len(boxes)
        num_images += 1
        
        if len(boxes):
            for i, bbox in enumerate(boxes):
                bbox = bbox.tolist()
                width = bbox[2] - bbox[0]
                if width < 1000:
                    crop_name = filename.replace("/", "-") + "_" + str(i)
                    cr = img.crop(tuple(bbox))
                    cr.save(crops_dir + crop_name, "JPEG")
                    table.append({
                        "file_name": filename,
                        "xmin": bbox[0],
                        "ymin": bbox[1],
                        "xmax": bbox[2],
                        "ymax": bbox[3],
                        "bbox_id": i,
                        "detection_confidence": confs[i],
                        "crop_name": crop_name
                    })
                    num_valid += 1
                else:
                    num_invalid += 1
        else:
            empty_images += 1

In [64]:
df = pd.DataFrame(table)
df.head(n=100).file_name.values

array(['rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.192507.02388_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.190755.02001_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.190755.02001_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.175042.00264_rect_color.jpg',
       'rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.175042.002

In [62]:
print("num_images", num_images)
print("num_boxes", num_boxes)

num_images 57468
num_boxes 421958


In [61]:
df.to_csv(crops_dir + "x152_crops.csv")